# Data Cleaning Tutorial with Python and Pandas

Welcome to this interactive tutorial on Data Cleaning! 

Data cleaning is a crucial step in any data science or machine learning project. Real-world data is often "messy"—it contains missing values, duplicates, errors, and inconsistencies. If we feed this messy data into our models, the results will be unreliable (Garbage In, Garbage Out).

In this notebook, we will cover:
1.  **Generating Synthetic Data**: Creating a messy dataset to practice on.
2.  **Data Inspection**: Understanding the structure and issues in your data.
3.  **Handling Missing Values**: Deciding whether to drop or fill missing data.
4.  **Removing Duplicates**: Identifying and deleting repeated entries.
5.  **Fixing Inconsistencies**: Standardizing text and categorical data.
6.  **Handling Outliers**: Detecting and managing extreme values.
7.  **Type Conversion**: Ensuring data types are correct for analysis.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Set a seed for reproducibility
np.random.seed(42)

print("Libraries imported successfully!")

## 1. Generating a "Messy" Dataset

To ensure everyone has the same data and to demonstrate specific cleaning challenges, we will generate a synthetic dataset representing a **Customer Database**.

Our dataset will intentionally include:
-   **Missing Values (`NaN`)**: Simulating incomplete forms.
-   **Duplicates**: Simulating data entry errors.
-   **Inconsistent Text**: e.g., "New York", "new york", "NY".
-   **Outliers**: e.g., an Age of 200.
-   **Wrong Data Types**: Numbers stored as strings.

In [ ]:
# Create a dictionary of data
data = {
    'CustomerID': [101, 102, 103, 104, 105, 106, 107, 108, 109, 101, 110, 111],
    'Name': ['John Doe', 'Jane Smith', 'Robert Brown', 'Emily Davis', 'Michael Wilson', 
             'Sarah Miller', 'David Garcia', 'Jennifer Martinez', 'James Anderson', 'John Doe', 'Lisa Thomas', 'Paul Jackson'],
    'Age': [28, 34, np.nan, 22, 45, 200, 31, np.nan, 29, 28, 39, 25],
    'City': ['New York', 'Los Angeles', 'Chicago', 'new york', 'Houston', 
             'New_York', 'Chicago', 'San Diego', 'Dallas', 'New York', 'Miami', 'NY'],
    'Salary': ['50000', '60000', '55000', '48000', '70000', '65000', '52000', '58000', '62000', '50000', '75000', '$54000'],
    'JoinDate': ['2020-01-15', '2019-05-20', '2021-02-10', '2020-01-15', '2018-11-05', 
                 '2021-06-30', '2019-09-12', '2020-08-25', '2021-01-01', '2020-01-15', '2017-03-18', '2020/12/12']
}

# Create DataFrame
df = pd.DataFrame(data)

# Display the first few rows
df.head(12)

## 2. Data Inspection

Before cleaning, we must inspect the data to identify issues. We use methods like `.info()`, `.describe()`, and `.isnull()`.

In [ ]:
# Check data types and non-null counts
print("--- Data Info ---")
df.info()

# Check for missing values
print("\n--- Missing Values ---")
print(df.isnull().sum())

**Observations:**
-   `Age` has missing values.
-   `Salary` is of type `object` (string) instead of integer/float, likely due to the '$' sign.
-   `City` has inconsistent naming (New York, new york, New_York, NY).

## 3. Handling Duplicates

Duplicate rows can skew analysis. We should identify and remove them.

In [ ]:
# Check for duplicates
duplicates = df.duplicated().sum()
print(f"Number of duplicate rows: {duplicates}")

# Remove duplicates
df = df.drop_duplicates()

# Verify
print(f"Number of duplicate rows after cleaning: {df.duplicated().sum()}")

## 4. Handling Missing Values

We have missing values in the `Age` column. We can either:
1.  **Drop** the rows with missing data (if we have a lot of data).
2.  **Impute** (fill) the missing data with the Mean, Median, or Mode.

Here, we will fill missing `Age` with the **median** age, as it is robust to outliers.

In [ ]:
# Fill missing Age with the median
median_age = df['Age'].median()
df['Age'] = df['Age'].fillna(median_age)

# Verify
print(df.isnull().sum())

## 5. Fixing Inconsistent Data (Standardization)

The `City` column has variations for "New York". We need to standardize them.

In [ ]:
# Check unique values
print("Cities before cleaning:", df['City'].unique())

# Standardize to lowercase and strip whitespace
df['City'] = df['City'].str.lower().str.strip()

# Replace variations
df['City'] = df['City'].replace({'new_york': 'new york', 'ny': 'new york'})

# Capitalize for display
df['City'] = df['City'].str.title()

print("Cities after cleaning:", df['City'].unique())

## 6. Data Type Conversion and Cleaning Strings

The `Salary` column is a string because of the '$' symbol in one entry. We need to remove it and convert the column to numeric.

In [ ]:
# Remove '$' and convert to numeric
df['Salary'] = df['Salary'].astype(str).str.replace('$', '', regex=False)
df['Salary'] = pd.to_numeric(df['Salary'])

print(df['Salary'].dtypes)

## 7. Handling Outliers

We noticed an `Age` of 200, which is impossible. This is an outlier. We can visualize it and then remove or cap it.

In [ ]:
# Visualize with a Boxplot
plt.figure(figsize=(8, 4))
sns.boxplot(x=df['Age'])
plt.title("Age Distribution (with Outlier)")
plt.show()

In [ ]:
# Remove rows where Age is greater than 100
df = df[df['Age'] <= 100]

# Verify
print("Max Age:", df['Age'].max())

## 8. Conclusion

We have successfully cleaned our dataset! 

**Summary of actions:**
1.  Removed duplicates.
2.  Filled missing values in `Age`.
3.  Standardized `City` names.
4.  Converted `Salary` to numeric.
5.  Removed impossible `Age` outliers.

The data is now ready for analysis or machine learning.

In [ ]:
# Final look at the clean data
df.reset_index(drop=True, inplace=True)


combined = pd.concat([df, main_df], axis=1)
display(combined)

# Data Preproccesing for LLM 

## Load Model and libraries

In [ ]:
!pip install transformers

In [ ]:
from transformers import AutoTokenizer
from difflib import SequenceMatcher

print("⏳ Loading Tokenizer (this may take a moment)...")
# We use a multilingual tokenizer to demonstrate sub-word splitting effectively
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")


## Tokenization

In [ ]:
# ==========================================
# PART A: Tokenization (The Engine of LLMs)
# ==========================================
print("\n" + "="*60)
print("PART A: Sub-word Tokenization Explained")
print("="*60)

# Examples: Simple English vs. Complex/Hyphenated words
texts = [
    "The AI model is fast.",           # Simple words
    "Unbelievably good results!",      # Complex suffix
    "Pre-processing is essential.",    # Hyphenated
    "Ryan_Heida"
]

for txt in texts:
    # 1. Tokenize text
    tokens = tokenizer.tokenize(txt)
    # 2. Convert to IDs (what the model actually sees)
    ids = tokenizer.convert_tokens_to_ids(tokens)
    
    print(f"\n[Input]:  {txt}")
    print(f"[Tokens]: {tokens}")
    print(f"[IDs]:    {ids}")
    print(f"[Count]:  {len(tokens)} tokens")

print("\n>>> KEY INSIGHT: Notice symbols like '##'. This is 'Sub-word Tokenization'.")
print(">>> Rare words are broken down into smaller chunks to save vocabulary space.")



## Duplication

In [ ]:
# ==========================================
# PART B: Deduplication (Quality Control)
# ==========================================
print("\n" + "="*60)
print("PART B: Data Deduplication (Exact vs. Fuzzy)")
print("="*60)

# Simulating three documents from a web crawl
doc_original = "Deep Learning uses neural networks with many layers."
doc_exact    = "Deep Learning uses neural networks with many layers."       # 100% Copy
doc_fuzzy    = "Deep Learning utilizes neural nets having multiple layers." # Semantic Copy

def get_similarity(a, b):
    return SequenceMatcher(None, a, b).ratio()

# 1. Exact Match Check
print(f"1. Exact Match Check (Original vs Copy): {'MATCH' if doc_original == doc_exact else 'NO MATCH'}")

# 2. Fuzzy Match Check (MinHash logic simulation)
sim_score = get_similarity(doc_original, doc_fuzzy)
print(f"2. Fuzzy Similarity Score: {sim_score:.2f} (Scale 0-1)")

# Decision Logic
dedup_threshold = 0.75
if sim_score > dedup_threshold:
    print(f">>> DECISION: DROP the fuzzy copy. (Too similar, risks data leakage).")
else:
    print(f">>> DECISION: KEEP the fuzzy copy.")



## Instruction Tuning

### My Template (For My Model)

In [ ]:

# ==========================================
# PART C: Instruction Tuning (Chat Templates)
# ==========================================
print("\n" + "="*60)
print("PART C: Formatting for Instruction Fine-Tuning")
print("="*60)

# Raw data from the internet
raw_fact = "The Eiffel Tower is located in Paris, France."


# My Chat Template
chat_template = """{% for message in messages -%}
{{ message.role }}: {{ message.content }}
{% endfor %}"""

# Converting raw data into a Conversational Format (ChatML style)
# This is required for training models like Llama 3, ChatGPT, etc.
chat_entry = [
    {"role": "system",    "content": "You are a helpful geography assistant."},
    {"role": "user",      "content": "Where is the Eiffel Tower?"},
    {"role": "assistant", "content": raw_fact}
]

# Apply the template to format the string
formatted_string = tokenizer.apply_chat_template(
    chat_entry,
    chat_template=chat_template,
    tokenize=False
)

print("--- Final Training Sample ---")
print(formatted_string)
print("-----------------------------")

### Use a model tokenizer

In [ ]:
# Login to HF

from huggingface_hub import login
login()  # then paste your token when prompted

In [ ]:

# ==========================================
# PART C: Instruction Tuning (Chat Templates)
# ==========================================
print("\n" + "="*60)
print("PART C: Formatting for Instruction Fine-Tuning")
print("="*60)

# Raw data from the internet
raw_fact = "The Eiffel Tower is located in Paris, France."

# Converting raw data into a Conversational Format (ChatML style)
# This is required for training models like Llama 3, ChatGPT, etc.
chat_entry = [
    {"role": "system",    "content": "You are a helpful geography assistant."},
    {"role": "user",      "content": "Where is the Eiffel Tower?"},
    {"role": "assistant", "content": raw_fact}
]


tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-7B-Instruct")

formatted_string = tokenizer.apply_chat_template(
    chat_entry,
    tokenize=False
)


print("--- Final Training Sample ---")
print(formatted_string)
print("-----------------------------")

* **`<|im_start|>`**: Marks the **start** of a speaker's turn.
* **`system`**: Defines the model's **persona/rules**.
* **`user`**: Contains the **user's instruction/query**.
* **`assistant`**: Contains the **target response** for the model to learn.
* **`<|im_end|>`**: Marks the **end** of a speaker's turn.